# Usando SQLite no python

In [ ]:
# importando o driver do sqlite
import sqlite3
from pathlib import Path

# Diretório do arquivo atual
#BASE_DIR = Path(__file__).resolve().parent
#print(BASE_DIR)

# Caminho completo do banco
#DB_PATH = BASE_DIR / "db" / "escola_v2.db"
#print(DB_PATH)

# inicia a conexao com o banco de dados.
db_connection = sqlite3.connect('db/escola_v2.db')

# cria o cursor
cursor = db_connection.cursor()

# executa qualquer comando SQL suportado pelo sqlite
cursor.execute("""
SELECT * 
FROM Aluno
""")

# Salva o resultado da query na variável
query_result = cursor.fetchall()

# Imprime o resultado da query
print(query_result)

# Encerra a conexao com o banco de dados
db_connection.close()

Note:
- O banco de dados é informado como parâmetro do método connect e armazenado na variável db_connection;
- Caso o banco não exista, um será criado com o mesmo nome passado como parâmetro do método connect;
- O cursor é um objeto que permite executar comandos SQL;
- O método fetchall, le todas as linhas/registros obtidos pela query. Existem ainda os métodos fetchone e fetchmany, onde o primeiro lê apenas o primeiro registro obtido pela query, já o fetchmany, le a quantidade de registros informado como parâmetro do método. ex: fetchmany(5) retorna 5 registros;

## Acessando os dados em lote

Realizar consultas e operações em lote pode ser muito útil para economizar recursos.

- Usando fetchmany

In [ ]:
import sqlite3

db_connection = sqlite3.connect("db/escola_v2.db")
cursor = db_connection.cursor()
cursor.execute("""
        SELECT * 
        FROM Aluno
        """)
rows = cursor.fetchmany(5)

while row:
    for row in rows:
        print(row)
    rows = cursor.fetchmany(5)
db_connection.close()

- Linha por linha

In [ ]:
import sqlite3

with sqlite3.connect("db/escola_v2.db") as db_connection:
    cursor = db_connection.cursor()
    cursor.execute("""
        SELECT * 
        FROM Aluno
        """)
    for row in cursor:
        print(row)
    
    #row = cursor.fetchone()
    #while row:
    #    print(row)
    #    row = cursor.fetchone()

db_connection.close()


## Testando a eficiencia

In [6]:
import sqlite3
import time
import tracemalloc

db_connection = sqlite3.connect("db/escola_v2.db")

cursor = db_connection.cursor()
cursor.execute("""
        SELECT * 
        FROM Aluno
        """)

# ---------- Teste 1: fetchall() ----------
print("Teste 1: fetchall()")

tracemalloc.start()
inicio = time.perf_counter()

cursor.execute("SELECT * FROM Aluno")
rows = cursor.fetchall()  # carrega tudo de uma vez

tempo_total = time.perf_counter() - inicio
memoria_atual, memoria_pico = tracemalloc.get_traced_memory()
tracemalloc.stop()

print(f"Tempo: {tempo_total:.4f}s")
print(f"Memória pico: {memoria_pico / 1024:.1f} KB\n")


# ---------- Teste 2: fetchmany(5) ----------
print("Teste 2: fetchmany(5)")

tracemalloc.start()
inicio = time.perf_counter()

cursor.execute("SELECT * FROM Aluno")
while True:
    rows = cursor.fetchmany(5)
    if not rows:
        break
    # Simula processamento
    for r in rows:
        _ = r[1]

tempo_total = time.perf_counter() - inicio
memoria_atual, memoria_pico = tracemalloc.get_traced_memory()
tracemalloc.stop()

print(f"Tempo: {tempo_total:.4f}s")
print(f"Memória pico: {memoria_pico / 1024:.1f} KB\n")

# ---------- Teste 3: Linha por linha ----------
print("Teste 3: Linha por linha")

tracemalloc.start()
inicio = time.perf_counter()

cursor.execute("SELECT * FROM Aluno")
while True:
    row = cursor.fetchone()
    if not row:
        break
    # Simula processamento
    _ = row

tempo_total = time.perf_counter() - inicio
memoria_atual, memoria_pico = tracemalloc.get_traced_memory()
tracemalloc.stop()

print(f"Tempo: {tempo_total:.4f}s")
print(f"Memória pico: {memoria_pico / 1024:.1f} KB\n")

db_connection.close()

Teste 1: fetchall()
Tempo: 0.0042s
Memória pico: 63.3 KB

Teste 2: fetchmany(5)
Tempo: 0.0037s
Memória pico: 26.8 KB

Teste 3: Linha por linha
Tempo: 0.0021s
Memória pico: 24.5 KB



In [7]:
import sqlite3
import time
import tracemalloc  # mede uso de memória


# Cria e popula banco
conn = sqlite3.connect("teste.db")
cursor = conn.cursor()

cursor.execute("DROP TABLE IF EXISTS alunos")
cursor.execute("CREATE TABLE alunos (id INTEGER PRIMARY KEY, nome TEXT, idade INTEGER)")

# Inserir 10.000 registros
cursor.executemany("INSERT INTO alunos (nome, idade) VALUES (?, ?)", [
    (f"Aluno {i}", 20 + (i % 10)) for i in range(10000)
])
conn.commit()
print("✅ Banco criado e populado com 10.000 registros.\n")


# ---------- Teste 1: fetchall() ----------
print("Teste 1: fetchall()")

tracemalloc.start()
inicio = time.perf_counter()

cursor.execute("SELECT * FROM alunos")
rows = cursor.fetchall()  # carrega tudo de uma vez

tempo_total = time.perf_counter() - inicio
memoria_atual, memoria_pico = tracemalloc.get_traced_memory()
tracemalloc.stop()

print(f"Tempo: {tempo_total:.4f}s")
print(f"Memória pico: {memoria_pico / 1024:.1f} KB\n")


# ---------- Teste 2: fetchmany(1000) ----------
print("Teste 2: fetchmany(1000)")

tracemalloc.start()
inicio = time.perf_counter()

cursor.execute("SELECT * FROM alunos")
while True:
    rows = cursor.fetchmany(1000)
    if not rows:
        break
    # Simula processamento
    for r in rows:
        _ = r[1]

tempo_total = time.perf_counter() - inicio
memoria_atual, memoria_pico = tracemalloc.get_traced_memory()
tracemalloc.stop()

print(f"Tempo: {tempo_total:.4f}s")
print(f"Memória pico: {memoria_pico / 1024:.1f} KB\n")


# ---------- Teste 3: linha por linha ----------
print("Teste 3: Linha por linha")

tracemalloc.start()
inicio = time.perf_counter()

cursor.execute("SELECT * FROM alunos")
while True:
    row = cursor.fetchone()
    if not row:
        break
    # Simula processamento
    _ = row

tempo_total = time.perf_counter() - inicio
memoria_atual, memoria_pico = tracemalloc.get_traced_memory()
tracemalloc.stop()

print(f"Tempo: {tempo_total:.4f}s")
print(f"Memória pico: {memoria_pico / 1024:.1f} KB\n")

conn.close()



✅ Banco criado e populado com 10.000 registros.

Teste 1: fetchall()
Tempo: 0.0509s
Memória pico: 1521.3 KB

Teste 2: fetchmany(1000)
Tempo: 0.0866s
Memória pico: 242.8 KB

Teste 3: Linha por linha
Tempo: 0.0864s
Memória pico: 24.4 KB

